In [11]:
import requests

In [12]:
import pandas as pd

In [72]:


#depa_list = [{"codigo_departamento":"04","departamento":"AREQUIPA"}]

#res = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/departamento/')
#res.encoding = 'utf-8'
#depa_list = res.json()

#depa_list = [{"codigo_departamento":"04","departamento":"AREQUIPA"}]
depa_list = [{"codigo_departamento":"01","departamento":"AMAZONAS"},
             {"codigo_departamento":"02","departamento":"ANCASH"},
             {"codigo_departamento":"03","departamento":"APURIMAC"},{"codigo_departamento":"04","departamento":"AREQUIPA"},
             {"codigo_departamento":"05","departamento":"AYACUCHO"},{"codigo_departamento":"06","departamento":"CAJAMARCA"},
             {"codigo_departamento":"07","departamento":"CALLAO"},{"codigo_departamento":"08","departamento":"CUSCO"},
             {"codigo_departamento":"09","departamento":"HUANCAVELICA"},{"codigo_departamento":"10","departamento":"HUANUCO"},
             {"codigo_departamento":"11","departamento":"ICA"},{"codigo_departamento":"12","departamento":"JUNIN"},
             {"codigo_departamento":"13","departamento":"LA LIBERTAD"},{"codigo_departamento":"14","departamento":"LAMBAYEQUE"},
             {"codigo_departamento":"15","departamento":"LIMA"},{"codigo_departamento":"16","departamento":"LORETO"},
             {"codigo_departamento":"17","departamento":"MADRE DE DIOS"},{"codigo_departamento":"18","departamento":"MOQUEGUA"},
             {"codigo_departamento":"19","departamento":"PASCO"},{"codigo_departamento":"20","departamento":"PIURA"},
             {"codigo_departamento":"21","departamento":"PUNO"},{"codigo_departamento":"22","departamento":"SAN MARTIN"},
             {"codigo_departamento":"23","departamento":"TACNA"},{"codigo_departamento":"24","departamento":"TUMBES"},
             {"codigo_departamento":"25","departamento":"UCAYALI"}]

def process_data(df):
    df['nombre_comercial']=df['nombre_comercial'].strip().upper()
    df['razon_social']=df['razon_social'].strip().upper()
    df['nombre_banco']=df['nombre_banco'].strip().upper()
    
    df['nombre_comercial'] = df['nombre_comercial'][1:] if df['nombre_comercial'][:1] == '-' else df['nombre_comercial']
    
    df['tipo_rubro_str'] = 'P' if df['tipo_rubro'] == 1 else 'B'
    
    return df
            
            
def get_depa(dep_list):
    
    columns = ["id_cuestionario_mype", "ruc", "latitud","longitud", "razon_social", "nombre_comercial", "es_agente", "nombre_banco",
           "vende_medicamento", "horario_atencion", "numero_documento", "nombres", "apellido_paterno", "apellido_materno", "correo",
           "telefono", "celular", "codigo_departamento", "codigo_provincia", "codigo_distrito", "direccion", "pago_tarjeta", 
           "pago_apps", "flag",  "stock_productos", "nombre_apps", "departamento", "provincia", "distrito", "tipo_rubro", "vende_perecibles",
           "vende_no_perecibles", "vende_higiene", "vende_aseo", "acepta_yape", "acepta_lukita", "acepta_tunki", "alimento_mascotas"] 
    
    #temp_df = pd.DataFrame([],columns=[])
    #list_resume = []
    temp_full = pd.DataFrame([],columns=[])
    temp_resume = pd.DataFrame([],columns=[])
    for depa in depa_list:
    
        id_depa = depa.get('codigo_departamento')
        name_depa = depa.get('departamento')
        print('DEPARTAMENTO > ',name_depa)

        #select provincia by departament
        res_pro = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/provincia/'+id_depa)
        res_pro.encoding = 'utf-8'
        prov_list = res_pro.json()

        #call get_prov
        list_resume = []
        full_df = pd.DataFrame([],columns=[])
        resume_columns = ['departamento','provincia','distrito','total_tiendas']
        resume_df = pd.DataFrame([],columns=resume_columns)
    
        full_df, list_resume = get_prov(prov_list, id_depa, name_depa)

        full_df = full_df.reset_index(drop=True, inplace=False)

        resume_df = pd.DataFrame(list_resume)

        
        
        #mayus
        full_df = full_df.apply(process_data,axis=1)
        
        
        name_depa = name_depa.lower()
        full_df.to_csv(name_depa+'_full.csv',index=False,header=True,encoding='utf-8')
        resume_df.to_csv(name_depa+'_resume.csv',index=False,header=True,encoding='utf-8')

        
        temp_full = pd.DataFrame([],columns=[])
        temp_resume = pd.DataFrame([],columns=[])
        
        temp_full = full_df
        temp_resume = resume_df
        print('CSV > ' + name_depa)
        
    return temp_full, temp_resume
    
def get_prov(prov_list, id_depa, name_depa):
    
    depa_df = pd.DataFrame([],columns=[])
    temp_df = pd.DataFrame([],columns=[])
    list_by_dis = []
    
    for prov in prov_list:
        id_prov = prov.get('codigo_provincia')
        name_prov = prov.get('provincia')
        print('PROVINCIA > ',name_prov)
        
        #select distrito by provincia by department
        extend = id_depa + '/' + id_prov
        res_dis = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/distritos/'+extend)
        res_dis.encoding = 'utf-8'
        dis_list = res_dis.json()
        
        list_code = [id_depa,id_prov]
        list_name = [name_depa, name_prov]
        temp_df, list_temp = get_dis(dis_list,list_code,list_name)
        depa_df = depa_df.append(temp_df)
        list_by_dis.extend(list_temp)
        
    return depa_df, list_by_dis
    
def get_dis(dis_list,entries_code,entries_name):
    
    
    str_entry = '/'.join(map(str,entries_code))
    prov_df = pd.DataFrame([],columns=[])
    temp_df = pd.DataFrame([],columns=[])
    
    list_resume = []
    temp_dict = {}
    
    for dis in dis_list:
        id_dis = dis.get('codigo_distrito')
        name_dis = dis.get('distrito')

        final_extend = str_entry + '/' + id_dis

        response = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/selecciona-bodega/'+final_extend)
        response.encoding = 'utf-8'
        lista_tiendas = response.json().get('data')

        temp_df, temp_dict = get_tiendas(lista_tiendas)
        prov_df = prov_df.append(temp_df)
        
        temp_dict['departamento'] = entries_name[0]
        temp_dict['provincia'] = entries_name[1]
        temp_dict['distrito'] = name_dis
        list_resume.append(temp_dict)
        temp_dict = {}
        
    return prov_df, list_resume
            
    
def get_tiendas(list_tiendas):
    dis_df = pd.DataFrame([],columns=[])
    temp_dict = {}
    num_bodegas = 0
    if list_tiendas != None:
        num_bodegas = len(list_tiendas)
        df = pd.DataFrame(list_tiendas)
        #filter_df = df[columns]
        dis_df = dis_df.append(df)
        
    temp_dict['total_tiendas'] = num_bodegas
    
    return dis_df, temp_dict    


In [73]:
full_, resume_ = get_depa(depa_list)

DEPARTAMENTO >  AMAZONAS
PROVINCIA >  BAGUA
PROVINCIA >  BONGARA
PROVINCIA >  CHACHAPOYAS
PROVINCIA >  CONDORCANQUI
PROVINCIA >  LUYA
PROVINCIA >  RODRIGUEZ DE MENDOZA
PROVINCIA >  UTCUBAMBA
CSV > amazonas
DEPARTAMENTO >  ANCASH
PROVINCIA >  AIJA
PROVINCIA >  ANTONIO RAYMONDI
PROVINCIA >  ASUNCION
PROVINCIA >  BOLOGNESI
PROVINCIA >  CARHUAZ
PROVINCIA >  CARLOS FERMIN FITZCARRALD
PROVINCIA >  CASMA
PROVINCIA >  CORONGO
PROVINCIA >  HUARAZ
PROVINCIA >  HUARI
PROVINCIA >  HUARMEY
PROVINCIA >  HUAYLAS
PROVINCIA >  MARISCAL LUZURIAGA
PROVINCIA >  OCROS
PROVINCIA >  PALLASCA
PROVINCIA >  POMABAMBA
PROVINCIA >  RECUAY
PROVINCIA >  SANTA
PROVINCIA >  SIHUAS
PROVINCIA >  YUNGAY
CSV > ancash
DEPARTAMENTO >  APURIMAC
PROVINCIA >  ABANCAY
PROVINCIA >  ANDAHUAYLAS
PROVINCIA >  ANTABAMBA
PROVINCIA >  AYMARAES
PROVINCIA >  CHINCHEROS
PROVINCIA >  COTABAMBAS
PROVINCIA >  GRAU
CSV > apurimac
DEPARTAMENTO >  AREQUIPA
PROVINCIA >  AREQUIPA
PROVINCIA >  CAMANA
PROVINCIA >  CARAVELI
PROVINCIA >  CASTILLA
P

## Join csv

In [75]:
depa_list = [{"codigo_departamento":"01","departamento":"AMAZONAS"},
             {"codigo_departamento":"02","departamento":"ANCASH"},
             {"codigo_departamento":"03","departamento":"APURIMAC"},{"codigo_departamento":"04","departamento":"AREQUIPA"},
             {"codigo_departamento":"05","departamento":"AYACUCHO"},{"codigo_departamento":"06","departamento":"CAJAMARCA"},
             {"codigo_departamento":"07","departamento":"CALLAO"},{"codigo_departamento":"08","departamento":"CUSCO"},
             {"codigo_departamento":"09","departamento":"HUANCAVELICA"},{"codigo_departamento":"10","departamento":"HUANUCO"},
             {"codigo_departamento":"11","departamento":"ICA"},{"codigo_departamento":"12","departamento":"JUNIN"},
             {"codigo_departamento":"13","departamento":"LA LIBERTAD"},{"codigo_departamento":"14","departamento":"LAMBAYEQUE"},
             {"codigo_departamento":"15","departamento":"LIMA"},{"codigo_departamento":"16","departamento":"LORETO"},
             {"codigo_departamento":"17","departamento":"MADRE DE DIOS"},{"codigo_departamento":"18","departamento":"MOQUEGUA"},
             {"codigo_departamento":"19","departamento":"PASCO"},{"codigo_departamento":"20","departamento":"PIURA"},
             {"codigo_departamento":"21","departamento":"PUNO"},{"codigo_departamento":"22","departamento":"SAN MARTIN"},
             {"codigo_departamento":"23","departamento":"TACNA"},{"codigo_departamento":"24","departamento":"TUMBES"},
             {"codigo_departamento":"25","departamento":"UCAYALI"}]

In [154]:
list_name = []
for i in depa_list:
    name = i.get('departamento').lower()
    list_name.append(name)

all_filenames = [name+'_full.csv' for name in list_name]
resume_filenames = [name+'_resume.csv' for name in list_name]
#all_filenames = all_filenames[:2]
#all_filenames
#combine all files in the list
combined_full = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_resume = pd.concat([pd.read_csv(f) for f in resume_filenames ])
#export to csv
#combined_csv.to_csv( "combined_full.csv", index=False, encoding='utf-8-sig')

In [155]:
def process_aux(df):
    df['departamento']=df['departamento'].strip().upper()
    df['provincia']=df['provincia'].strip().upper()
    df['distrito']=df['distrito'].strip().upper()
    return df

combined_full = combined_full.apply(process_aux,axis=1)

In [156]:
combined_resume = combined_resume[['departamento','provincia','distrito','total_tiendas']]

In [162]:
combined_resume.to_csv( "combined_resume.csv", index=False, encoding='utf-8-sig')
combined_full.to_csv( "combined_full.csv", index=False, encoding='utf-8-sig')

### Validating data

In [110]:
combined_resume[['departamento','provincia','distrito']].describe()

,departamento,provincia,distrito
count,1868,1868,1868
unique,25,196,1709
top,LIMA,LIMA,SANTA ROSA
freq,169,43,10


In [128]:
un_resume = combined_resume['provincia'].unique()

In [163]:
df_temp = combined_resume.groupby(['departamento','provincia']).sum()
df_temp[df_temp['total_tiendas']==0]

total_tiendas
departamento provincia                          
AMAZONAS     BAGUA                             0
             LUYA                              0
             RODRIGUEZ DE MENDOZA              0
ANCASH       AIJA                              0
             ANTONIO RAYMONDI                  0
...                                          ...
SAN MARTIN   PICOTA                            0
TACNA        CANDARAVE                         0
             TARATA                            0
TUMBES       ZARUMILLA                         0
UCAYALI      PURUS                             0

[78 rows x 1 columns]

In [111]:
combined_full[['departamento','provincia','distrito']].describe()

,departamento,provincia,distrito
count,3116,3116,3116
unique,25,118,322
top,Lima,Lima,San juan de lurigancho
freq,1742,1682,155


In [135]:
combined_full = combined_full.apply(process_aux,axis=1)
un_full = combined_full['provincia'].unique()

In [136]:
list_zero = []
for i in un_resume:
    if i not in un_full:
        list_zero.append(i)

In [138]:
list_zero

['BAGUA',
 'LUYA',
 'RODRIGUEZ DE MENDOZA',
 'AIJA',
 'ANTONIO RAYMONDI',
 'ASUNCION',
 'HUARI',
 'MARISCAL LUZURIAGA',
 'OCROS',
 'PALLASCA',
 'POMABAMBA',
 'RECUAY',
 'SIHUAS',
 'YUNGAY',
 'ANTABAMBA',
 'AYMARAES',
 'CHINCHEROS',
 'COTABAMBAS',
 'GRAU',
 'CAMANA',
 'LA UNION',
 'HUANCA SANCOS',
 'HUANTA',
 'LA MAR',
 'LUCANAS',
 'PARINACOCHAS',
 'PAUCAR DEL SARA SARA',
 'VICTOR FAJARDO',
 'CONTUMAZA',
 'HUALGAYOC',
 'SAN MARCOS',
 'SAN MIGUEL',
 'SAN PABLO',
 'SANTA CRUZ',
 'ACOMAYO',
 'PARURO',
 'PAUCARTAMBO',
 'ACOBAMBA',
 'CASTROVIRREYNA',
 'CHURCAMPA',
 'HUAYTARA',
 'HUACAYBAMBA',
 'LAURICOCHA',
 'MARAÑON',
 'PUERTO INCA',
 'YAROWILCA',
 'JUNIN',
 'BOLIVAR',
 'CHEPEN',
 'GRAN CHIMU',
 'JULCAN',
 'PATAZ',
 'SANTIAGO DE CHUCO',
 'CANTA',
 'DATEM DEL MARAÑON',
 'LORETO',
 'MARISCAL RAMON CASTILLA',
 'PUTUMAYO',
 'REQUENA',
 'UCAYALI',
 'MANU',
 'GENERAL SANCHEZ CERRO',
 'DANIEL ALCIDES CARRION',
 'AYABACA',
 'AZANGARO',
 'HUANCANE',
 'LAMPA',
 'MELGAR',
 'MOHO',
 'SAN ANTONIO DE PUT

In [ ]:
combined_csv = combined_csv[]

In [104]:
combined_csv.shape

(3116, 39)

In [134]:
def process_aux(df):
    df['departamento']=df['departamento'].strip().upper()
    df['provincia']=df['provincia'].strip().upper()
    df['distrito']=df['distrito'].strip().upper()
    
    #df['nombre_comercial'] = df['nombre_comercial'][1:] if df['nombre_comercial'][:1] == '-' else df['nombre_comercial']
    
    #df['tipo_rubro_str'] = 'P' if df['tipo_rubro'] == 1 else 'B'
    
    return df
#combined_csv = combined_csv.apply(process_aux,axis=1)
#combined_csv.to_csv( "combined_resume.csv", index=False, encoding='utf-8-sig')

In [108]:
#full_data
#combined_csv = combined_csv[['departamento','provincia','distrito','total_tiendas']]
combined_csv[['departamento','provincia','distrito']].describe()

,departamento,provincia,distrito
count,3116,3116,3116
unique,25,118,322
top,LIMA,LIMA,SAN JUAN DE LURIGANCHO
freq,1742,1682,155


In [102]:
combined_csv[['departamento','provincia','distrito']].describe()

,departamento,provincia,distrito
count,1868,1868,1868
unique,25,196,1709
top,LIMA,LIMA,SANTA ROSA
freq,169,43,10


### testing

In [3]:

# REGLA : TODOS LOS CAMPOS CON - reemplazar con ""
#

columns = ['departamento','provincia','distrito','ruc','razon_social','nombre_comercial','latitud','longitud']
full_df = df = pd.DataFrame([],[])
#dis_list = [{"codigo_distrito":"02","distrito":"EL CENEPA"},{"codigo_distrito":"01","distrito":"NIEVA"},{"codigo_distrito":"03","distrito":"RIO SANTIAGO"}]
dis_list = [{"codigo_distrito":"02","distrito":"ACHOMA"},{"codigo_distrito":"03","distrito":"CABANACONDE"},{"codigo_distrito":"04","distrito":"CALLALLI"},{"codigo_distrito":"05","distrito":"CAYLLOMA"},{"codigo_distrito":"01","distrito":"CHIVAY"},{"codigo_distrito":"06","distrito":"COPORAQUE"},{"codigo_distrito":"07","distrito":"HUAMBO"},{"codigo_distrito":"08","distrito":"HUANCA"},{"codigo_distrito":"09","distrito":"ICHUPAMPA"},{"codigo_distrito":"10","distrito":"LARI"},{"codigo_distrito":"11","distrito":"LLUTA"},{"codigo_distrito":"12","distrito":"MACA"},{"codigo_distrito":"13","distrito":"MADRIGAL"},{"codigo_distrito":"20","distrito":"MAJES"},{"codigo_distrito":"14","distrito":"SAN ANTONIO DE CHUCA"},{"codigo_distrito":"15","distrito":"SIBAYO"},{"codigo_distrito":"16","distrito":"TAPAY"},{"codigo_distrito":"17","distrito":"TISCO"},{"codigo_distrito":"18","distrito":"TUTI"},{"codigo_distrito":"19","distrito":"YANQUE"}]
for dis in dis_list:
    id_dis = dis.get('codigo_distrito')
    name_dis = dis.get('distrito')
    
    final_extend = '04' + '/' + '05' + '/' + id_dis
    response = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/selecciona-bodega/'+final_extend)
    lista_bodegas = response.json().get('data')
    num_bodegas = 0
    if lista_bodegas != None:
        num_bodegas = len(lista_bodegas)
        df = pd.DataFrame(lista_bodegas)
        #filter_df = df[columns]
        full_df = full_df.append(filter_df)
        
    print(name_dis, num_bodegas)

full_df = full_df.reset_index(drop=True, inplace=False)
full_df

ACHOMA 0
CABANACONDE 0
CALLALLI 0
CAYLLOMA 0
CHIVAY 1
COPORAQUE 0
HUAMBO 0
HUANCA 0
ICHUPAMPA 0
LARI 0
LLUTA 0
MACA 0
MADRIGAL 0
MAJES 1
SAN ANTONIO DE CHUCA 0
SIBAYO 0
TAPAY 0
TISCO 0
TUTI 0
YANQUE 0


,departamento,provincia,distrito,ruc,razon_social,nombre_comercial,latitud,longitud
0,Arequipa,Caylloma,Chivay,10306423113,Cacya mamani celso doroteo,SLINA,-14.7923394,-71.4078453
1,Arequipa,Caylloma,Majes,10475554201,CABANILLAS SERRANO AMANDA ELIZABETH,PRODUCTOS DE LIM. CLEAN 4 YOU,-16.4359692,-71.6050339


In [6]:
res = requests.get('https://www.tuempresa.gob.pe/ubicatubodega/public/ajax/selecciona-bodega/15/01/22')
res.encoding = 'utf-8'
data = res.json()
data

{'data': [{'id_cuestionario_mype': 7351,
   'ruc': '10078267742',
   'latitud': '-12.124886',
   'longitud': '-77.032022',
   'razon_social': 'CUSI SARMIENTO DE PUCLLAS LEONARDA',
   'nombre_comercial': 'MINIMARKET LALO?S',
   'es_agente': 'NO',
   'nombre_banco': '',
   'vende_medicamento': 'SI',
   'horario_atencion': '9am - 7pm',
   'numero_documento': '40854008',
   'nombres': 'JACKELYN JANNET',
   'apellido_paterno': 'PUCLLAS',
   'apellido_materno': 'CUSI',
   'correo': 'jackelynjannet@hotmail.com',
   'telefono': '',
   'celular': '951307688',
   'codigo_departamento': '15',
   'codigo_provincia': '01',
   'codigo_distrito': '22',
   'direccion': 'Calle Porta 284',
   'pago_tarjeta': 'SI',
   'pago_apps': 'SI',
   'flag': 1,
   'stock_productos': None,
   'nombre_apps': 'Yape, Lukita',
   'departamento': 'Lima',
   'provincia': 'Lima',
   'distrito': 'Miraflores',
   'tipo_rubro': 0,
   'vende_perecibles': 'SI',
   'vende_no_perecibles': 'SI',
   'vende_higiene': 'SI',
   'vende

In [74]:
df = pd.DataFrame(data.get('data'))

NameError: name 'data' is not defined

In [11]:
df.to_csv('test.csv',index=False,header=True,encoding='utf-8')

In [10]:
datos = [{'id_cuestionario_mype': 7351,
   'ruc': '10078267742',
   'latitud': '-12.124886'},{'id_cuestionario_mype': 7351,
   'ruc': '10078267742',
   'latitud': '-12.124886',
    'longitud':''}]

dframe = pd.DataFrame(datos)
dframe

,id_cuestionario_mype,ruc,latitud,longitud
0,7351,10078267742,-12.124886,NaN
1,7351,10078267742,-12.124886,
